Setup

In [1]:
# Import dependencies
import openai
import json
import pandas as pd
import os
import requests

# Set API key
API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = os.getenv('OPENAI_API_KEY')

First look at the Python package

In [2]:
# Extract models available to DF, there's 64
models = openai.Model.list()
models = pd.DataFrame(models['data']).sort_values(by='id')

In [3]:
# Moderation assessment model

moderation_resp = openai.Moderation.create(model='text-moderation-latest', input='People from whoville are fools')
# moderation_resp = openai.Moderation.create(input='Wilder sent crashing to the canvas')
# moderation_resp = openai.Moderation.create(input='Knife crime')

# This is where the model results are stored
moderation_resp

<OpenAIObject id=modr-748OEMwCBQdnmum5YvqZueiwYu6d9 at 0x1a128086ed0> JSON: {
  "id": "modr-748OEMwCBQdnmum5YvqZueiwYu6d9",
  "model": "text-moderation-004",
  "results": [
    {
      "categories": {
        "hate": true,
        "hate/threatening": false,
        "self-harm": false,
        "sexual": false,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false
      },
      "category_scores": {
        "hate": 0.8631312847137451,
        "hate/threatening": 1.6965859117590298e-07,
        "self-harm": 1.101324365038181e-08,
        "sexual": 1.4872586007186328e-07,
        "sexual/minors": 1.4105261403329905e-09,
        "violence": 5.8814286603592336e-05,
        "violence/graphic": 2.6809404474192888e-08
      },
      "flagged": true
    }
  ]
}

In [4]:
# Completion model

completion_resp = openai.Completion.create(
    model='text-davinci-003',
    prompt='Complete the following phrase: Veni, Vidi,',
    temperature=0.7)

# This is where the model results are stored
completion_resp['choices'][0]['text']

' \n\nVici'

Now execute the same models but interacting directly with the API

In [5]:
# Define endpoint
url = 'https://api.openai.com/v1/moderations'

# Set headers including API key
headers = {
  'Authorization': f'Bearer {API_KEY}',
  'Content-Type': 'application/json'
}

# Set model inputs
payload = {
  'input': 'People from whoville are fools',
  'model': 'text-moderation-stable'
}

# Execute request
response = requests.post(url, headers=headers, json=payload)
response = json.loads(response.text)

response

{'id': 'modr-748OGU3Ww3TbkCL1XFM2yq2rT47aE',
 'model': 'text-moderation-001',
 'results': [{'flagged': True,
   'categories': {'sexual': False,
    'hate': True,
    'violence': False,
    'self-harm': False,
    'sexual/minors': False,
    'hate/threatening': False,
    'violence/graphic': False},
   'category_scores': {'sexual': 0.0032732095569372177,
    'hate': 0.8229409456253052,
    'violence': 0.014659680426120758,
    'self-harm': 0.0012743014376610518,
    'sexual/minors': 0.004013063386082649,
    'hate/threatening': 0.0007698708213865757,
    'violence/graphic': 0.0012585219228640199}}]}

In [6]:
# Define endpoint
url = 'https://api.openai.com/v1/completions'

# Set headers including API key
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

# Set model inputs
payload = {
    'prompt': 'Complete the following phrase: Veni, Vidi, ',
    'model': 'text-davinci-003',
    'max_tokens': 7,
    'temperature': 0.7
}

# Execute request
response = requests.post(url, headers=headers, json=payload)
response = json.loads(response.text)

# This is where the model results are stored
response['choices'][0]['text']

'\n\nVici'